In [2]:
!pip install --quiet -U langchain langchain-community langchain_postgres langchain-aws langchain_core

import boto3
import langchain
# import psycopg2
import sqlalchemy
from sqlalchemy import Column, Integer, Text, create_engine
from langchain.embeddings import BedrockEmbeddings
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain_aws import BedrockLLM
# from langchain_community.vectorstores.pgvector import PGVector, DistanceStrategy
from langchain_postgres.vectorstores import PGVector
import pandas
from io import StringIO
from sqlalchemy import create_engine, text
from langchain_core.prompts import PromptTemplate
import re
import time
import math

# meta.llama3-8b-instruct-v1:0, meta.llama3-70b-instruct-v1:0
# mistral.mistral-7b-instruct-v0:2, mistral.mistral-large-2402-v1:0
LLAMA_3_8B = "meta.llama3-8b-instruct-v1:0"
LLAMA_3_70B = "meta.llama3-70b-instruct-v1:0"
MISTRAL_7B = "mistral.mistral-7b-instruct-v0:2"
MISTRAL_LARGE = "mistral.mistral-large-2402-v1:0"

print("Installed and imported dependencies!")

Installed and imported dependencies!


In [3]:
# Get CSV from S3 Bucket
def get_csv_from_bucket(bucket_name, file_path):

    # Create a boto3 client
    s3_client = boto3.client('s3')

    # Fetch the CSV file from S3
    csv_file = s3_client.get_object(Bucket=bucket_name, Key=file_path)
    csv_content = csv_file['Body'].read().decode('utf-8')

    # Load the CSV content into a pandas DataFrame
    df = pandas.read_csv(StringIO(csv_content))

    # Display the DataFrame
    df.head()
    
    return df

# Update CSV file on server
def update_csv_server_side(df, server_file_path):
    df.to_csv(server_file_path, index=False)

# Check if CSV from bucket is the same as CSV on server
def check_same_csv(df1, df2):
    return df1.equals(df2)

bucket_name = 'bucket-name'
file_path = 'file-path-of-CSV-file'
server_file_path = 'whatever-you-want-to-call-and-store-the-CSV-to'
update_embeddings = False

# Load the CSV file from S3
new_data = get_csv_from_bucket(bucket_name, file_path)
print("Data loaded from S3:")
print(new_data.head())

# Load the existing CSV file from the server
try:
    existing_data = pandas.read_csv(server_file_path)
    print("Data loaded from server:")
    print(existing_data.head())
except FileNotFoundError:
    print("No existing CSV file on the server. It will be created.")
    existing_data = None

# Check if the new CSV file matches the existing one
if existing_data is not None and check_same_csv(new_data, existing_data):
    print("The CSV files are identical. Skipping update and embedding process.")
else:
    print("The CSV files are different. Updating the server file.")
    update_csv_server_side(new_data, server_file_path)
    update_embeddings = True

Data loaded from S3:
   Unnamed: 0  doc_id                                                url  \
0           0       0                  https://vancouver.calendar.ubc.ca   
1           1       1  https://vancouver.calendar.ubc.ca/dates-and-de...   
2           2       2  https://vancouver.calendar.ubc.ca/dates-and-de...   
3           3       3  https://vancouver.calendar.ubc.ca/dates-and-de...   
4           4       4  https://vancouver.calendar.ubc.ca/dates-and-de...   

   parent                                             titles  \
0     NaN                                                 []   
1     0.0                            ['Dates and Deadlines']   
2     1.0              ['Dates and Deadlines', 'Term Dates']   
3     2.0  ['Dates and Deadlines', 'Term Dates', 'Term Da...   
4     2.0  ['Dates and Deadlines', 'Term Dates', 'Term Da...   

             parent_titles                                               text  \
0                       []                              

In [ ]:
# Define the collection name for storing embeddings
COLLECTION_NAME = "rag_test"

CONNECTION_STRING = (
    "your-database-parameters"
)

print("Connection string:", CONNECTION_STRING)

In [5]:
# Initialize the text embedding model
embeddings = BedrockEmbeddings()

# Initialize the PGVector instance
db = PGVector(
    connection=CONNECTION_STRING,
    collection_name=COLLECTION_NAME,
    embeddings=embeddings
)

# Initialize the engine
engine = create_engine(CONNECTION_STRING)

# Create PGVector extension
with engine.connect() as connection:
    connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))

# Function to clear the table data
def clear_table_data(table_name):
    clear_table_query = f"DROP TABLE {table_name} CASCADE;"

    try:
        with engine.connect() as connection:
            connection.execute(text(clear_table_query))
            connection.commit()
            print(f"Cleared data from table '{table_name}'.")

            # Need to initialize PGVector instance again to create collection and embeddings table
            db = PGVector(
                connection=CONNECTION_STRING,
                collection_name=COLLECTION_NAME,
                embeddings=embeddings
            )
    except:
        print(f"Error deleting data from table '{table_name}'.")

# Function to insert embeddings in batches
def insert_embeddings_in_batches(db, documents, batch_size=500):
    for i in range(0, len(documents), batch_size):
        batch_documents = documents[i:i+batch_size]
        db.add_documents(
            documents=batch_documents
        )
        print(f"Inserted batch {i//batch_size + 1} of {len(documents)//batch_size + 1}")

# Function to check if a table exists and count the number of rows
def check_table_and_count_rows(table_name):

    # SQL query to check if the table exists
    table_exists_query = f"""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE table_name = '{table_name}'
    );
    """

    # SQL query to count the number of rows in the table
    row_count_query = f"SELECT COUNT(*) FROM {table_name};"
    
    with engine.connect() as connection:
        # Check if the table exists
        table_exists_result = connection.execute(text(table_exists_query))
        table_exists = table_exists_result.scalar()

        if table_exists:
            # Count the number of rows if the table exists
            row_count_result = connection.execute(text(row_count_query))
            row_count = row_count_result.scalar()
            print(f"The table '{table_name}' exists and contains {row_count} rows.")
            return row_count
        else:
            print(f"The table '{table_name}' does not exist.")
            return False

# Check if CSV file was updated
if update_embeddings:

    print("CSV changed. Need to update CSV.")
    # Set update_embeddings flag to False to not update on next run
    update_embeddings = False
    
    # If embeddings already exist, tables need to be cleared
    if check_table_and_count_rows('langchain_pg_collection') and check_table_and_count_rows('langchain_pg_embedding'):
        print("Embeddings already exist and need to be cleared")
        # Clear existing table data
        clear_table_data('langchain_pg_embedding')
        clear_table_data('langchain_pg_collection')

        # Check data in tables after deletion of previous data
        check_table_and_count_rows('langchain_pg_collection')
        check_table_and_count_rows('langchain_pg_embedding')

    # Create a CSV loader and load the data
    loader = CSVLoader(file_path=server_file_path)

    # Initialize a text splitter for dividing text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)

    # Load and split the document
    data = loader.load_and_split(text_splitter=text_splitter)

    # Insert embeddings in batches
    print("Inserting new embeddings in batches.")
    insert_embeddings_in_batches(db, data)
else:
    print("No update to CSV. No need to update embeddings.")
    

# langchain_pg_embedding table has embedding column with type USER-DEFINED, need to change this to VECTOR type with dimensions
try:
    with engine.connect() as connection:
        alter_table_query_1 = """
        ALTER TABLE langchain_pg_embedding
        ADD COLUMN embedding_vector VECTOR(1536);

        UPDATE langchain_pg_embedding
        SET embedding_vector = embedding;

        ALTER TABLE langchain_pg_embedding
        DROP COLUMN embedding;

        ALTER TABLE langchain_pg_embedding
        RENAME COLUMN embedding_vector TO embedding;
        """
        connection.execute(text(alter_table_query_1))
        connection.commit()
        print("Altered embeddings column in langchain_pg_embedding table")
except:
    print("Error altering embeddings column")

# Calculate the index parameters according to best practices
num_records = check_table_and_count_rows('langchain_pg_embedding')
num_lists = num_records / 1000
if num_lists < 10:
    num_lists = 10
if num_records > 1000000:
    num_lists = math.sqrt(num_records)

# Create index on embeddings table
try:
    with engine.connect() as connection:
        # create_index_query = f"""
        # CREATE INDEX ON langchain_pg_embedding USING ivfflat (embedding vector_l2_ops) WITH (lists = {num_lists});
        # """
        create_index_query = f"""
        CREATE INDEX ON langchain_pg_embedding USING hnsw (embedding vector_l2_ops);
        """
        connection.execute(text(create_index_query))
        connection.commit()
        print("Index created successfully")
except:
    # Close connection to RDS if error
    engine.dispose()

# Check data in tables
check_table_and_count_rows('langchain_pg_collection')
check_table_and_count_rows('langchain_pg_embedding')

# Close connection to RDS
engine.dispose()

print("Database initialized and populated.")

No update to CSV. No need to update embeddings.
Altered embeddings column in langchain_pg_embedding table
The table 'langchain_pg_embedding' exists and contains 15933 rows.
Index created successfully
The table 'langchain_pg_collection' exists and contains 1 rows.
The table 'langchain_pg_embedding' exists and contains 15933 rows.
Database initialized and populated.


In [6]:
# Function to list all indexes for a table
def list_indexes_for_table(table_name):
    list_indexes_query = f"""
    SELECT indexname
    FROM pg_indexes
    WHERE tablename = '{table_name}';
    """
    
    with engine.connect() as connection:
        result = connection.execute(text(list_indexes_query))
        indexes = result.fetchall()
        if indexes:
            print(f"Indexes on table '{table_name}':")
            for index in indexes:
                print(f"- {index[0]}")
        else:
            print(f"No indexes found on table '{table_name}'.")
        return indexes

# List all indexes for 'langchain_pg_embedding' table
list_indexes_for_table('langchain_pg_embedding')

Indexes on table 'langchain_pg_embedding':
- langchain_pg_embedding_pkey1
- ix_cmetadata_gin
- ix_langchain_pg_embedding_id
- langchain_pg_embedding_embedding_idx


[('langchain_pg_embedding_pkey1',),
 ('ix_cmetadata_gin',),
 ('ix_langchain_pg_embedding_id',),
 ('langchain_pg_embedding_embedding_idx',)]

In [7]:
# Function to get details of an index
def get_index_details(index_name):
    index_details_query = f"""
    SELECT indexname, indexdef
    FROM pg_indexes
    WHERE indexname = '{index_name}';
    """
    
    with engine.connect() as connection:
        result = connection.execute(text(index_details_query))
        index_details = result.fetchone()
        if index_details:
            print(f"Details of index '{index_name}':")
            print(f"Index Name: {index_details[0]}")
            print(f"Index Definition: {index_details[1]}")
        else:
            print(f"No details found for index '{index_name}'.")

# Verify details of the created index
get_index_details('langchain_pg_embedding_embedding_idx')

Details of index 'langchain_pg_embedding_embedding_idx':
Index Name: langchain_pg_embedding_embedding_idx
Index Definition: CREATE INDEX langchain_pg_embedding_embedding_idx ON public.langchain_pg_embedding USING hnsw (embedding vector_l2_ops)


In [23]:
def check_if_document_relates(document, question, answer, llm):

    # Set up the retrieval chain with the language model and database retriever
    related_chain = RetrievalQA.from_chain_type(
                                        llm=llm,
                                        retriever=db.as_retriever()
                                    )

    # Initializing prompt to be passed to chain
    prompt = f"""Here is a queston that a user asked you: {question} and here is your answer: {answer}.
                Here is the text from the source document you returned: {document["text"]}.
                Can you determine if this text relates with the question and helped answer it? Please return a concise answer.
                """

    if llm.model_id == LLAMA_3_8B:
        prompt = (
            "<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
            "You are an AI tasked with determining the relevance of documents to specific queries. Provide only the relevance information and avoid any unrelated information or questions.<|eot_id|>"
            f"<|start_header_id|>user<|end_header_id|>Query: {question}\n"
            f"Document: {document['text']}<|eot_id|>"
            "<|start_header_id|>assistant<|end_header_id|>Does the document relate to the query? Provide a detailed explanation.<|eot_id|>"
        )
    elif llm.model_id == LLAMA_3_70B:
        prompt = (
            "<|startoftext|><|user|>"
            f"Query: {question}\n"
            f"Document: {document['text']}\n"
            "<|assistant|>Does the document relate to the query? Provide only the relevance information and avoid any unrelated information or questions.<|endoftext|>"
        )
    # elif llm.model_id == MISTRAL_7B:
    #     prompt = (
    #         f"### Instruction: Does the document relate to the query? Answer with 'yes' or 'no.'\n"
    #         f"Query: {question}\n"
    #         f"Document: {document['text']}\n"
    #         "### Answer: "
    #     )

    # Hold the result generated from the LLM
    result = related_chain.invoke(prompt)

    return result["result"]

def answer_prompt(question, llm):

    # Record the start times
    total_start_time = time.time()
    answer_start_time = time.time()

    llm = BedrockLLM(
                    model_id=llm
                )

    # Set up the retrieval chain with the language model and database retriever
    chain = RetrievalQA.from_chain_type(
                                            llm=llm,
                                            retriever=db.as_retriever(),
                                            return_source_documents=True
                                        )

    # Initializing prompt to be passed to chain
    prompt = question

    if llm.model_id == LLAMA_3_8B:
        prompt = (
            "<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
            "You are a helpful UBC student advising assistant who answers with kindness while being concise. Only generate one human readable answer"
            f"<|start_header_id|>user<|end_header_id|>{question}"
            "<|start_header_id|>assistant<|end_header_id|>"
        )
    elif llm.model_id == LLAMA_3_70B:
        prompt = (
            "<|startoftext|><|user|>"
            "You are a helpful UBC student advising assistant who answers with kindness."
            f"{question}\n"
            "<|assistant|>"
        )
    # elif llm.model_id.startswith("mistral."):
    #     prompt = (
    #         f"### Instruction:\nYou are a helpful UBC student advising assistant.\n\n"
    #         f"### Query: {question}\n"
    #         "### Answer:"
    #     )

    # Hold the result generated from the LLM
    result = chain.invoke(prompt)

    # Record the end time and find duration of answer only
    answer_end_time = time.time()
    answer_duration = answer_end_time - answer_start_time

    # Record query and result
    result_query = question
    result_answer = result["result"].strip()

    # Initialize data structure to hold sources
    source_documents = []

    # Loop through each document in the source documents. Maximum of 5 documents to avoid latency
    for doc in result['source_documents'][:5]:
        # Extract the 'url' value from the page content
        url_match = re.search(r'url:\s*(\S+)', doc.page_content)

        # Extract the 'text' value from the page content
        text_match = re.search(r'text:\s*((.|\n)*?)(?=\n\w+:\s|$)', doc.page_content)

        # Skip this iteration if either url or text is not found
        if not url_match or not text_match:
            continue

        url = url_match.group(1)
        text = text_match.group(1)

        # Check if text found has large enough size
        if len(text) > 20:
            # Create a dictionary with the 'url' and 'text' values
            source_dict = {"url": url, "text": text}

            # Get answer to whether the text in the source relates with the question and answer
            doc_relate = check_if_document_relates(source_dict, result_query, result_answer, llm)

            # Add doc_relate to dictionary
            source_dict["doc_relate"] = doc_relate

            # Append the dictionary to the list
            source_documents.append(source_dict)

    # Record the end time and find duration of the total time of checking over each document
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time

    # Return hashmap that holds query, result, source_documents, and time
    return {"query": result_query, "result": result_answer, "source_documents": source_documents, "answer_time": answer_duration, "total_time": total_duration}

# Choose between LLAMA_3_8B, LLAMA_3_70B, MISTRAL_7B, MISTRAL_LARGE
response = answer_prompt("Does physics 100 count for the arts requirement?", LLAMA_3_8B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: Does physics 100 count for the arts requirement? 

result: According to the UBC calendar, Physics 100 is a Science course, so it wouldn't count towards the Arts requirement. You can fulfill the Arts requirement by taking courses in the humanities, social sciences, and languages. If you have any more questions or need further clarification, feel free to ask! 

source_documents: [{'url': 'https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-arts/bachelor-arts/registration#22590', 'text': 'Students are strongly advised to consult the Arts Undergraduate website and to seek advising from the appropriate department or unit when selecting or modifying their course load.', 'doc_relate': ' No'}, {'url': 'https://science.ubc.ca/students/minors', 'text': 'Students should design a course of studies for their proposed Arts Minor, which must be approved by Science Advising by the end of Year 3.18-credit Minor in Arts  \nStudents must submit an application to Science Advisi

In [16]:
response = answer_prompt("Can I add a minor in fourth year?", LLAMA_3_8B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: Can I add a minor in fourth year? 

result: Unfortunately, it's generally not possible to add a minor in fourth year. The Science Advising team recommends exploring program options and career paths earlier in your undergraduate degree. However, I recommend reaching out to the Science Advising team or your program advisor to discuss your specific situation and potential alternatives. 

source_documents: [{'url': 'https://science.ubc.ca/students/advising', 'text': "1. Join the Science Advising line. \n  2. Review the final confirmation screen carefully.\n  3. When it's your turn, you'll get a text with instructions.", 'doc_relate': ' No, the document does not relate to the query. The document appears to be about the process of joining a Science Advising line, whereas the query is about adding a minor in fourth year.'}, {'url': 'https://science.ubc.ca/students/advising', 'text': "If a medical or personal issue is affecting your academic life see us as soon as possible! Don't wait f

In [11]:
response = answer_prompt("When do I have to apply to add a commerce minor?", LLAMA_3_70B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: When do I have to apply to add a commerce minor? 

result: I'm happy to help you with that! According to the UBC website, the application process and deadlines for adding a minor vary depending on your faculty and program. Unfortunately, I don't see that information in the provided context. I recommend checking the UBC website or contacting the Commerce department directly for the most up-to-date and accurate information. They'll be able to guide you through the process and provide you with the specific deadlines you need to know. Good luck with your application! 

source_documents: [{'url': 'https://science.ubc.ca/students/specialization-faq', 'text': 'Yes. If you’re waiting for an outcome regarding your UBC application, you still need to apply to the UBC BSc Specialization Application. If you’re offered admission to UBC Science, it’s an important step you need to have completed.', 'doc_relate': ' No, the document does not relate to the query.'}, {'url': 'https://science.ubc.ca

In [14]:
response = answer_prompt("I have an average of 54% and I have failed two courses this term. I haven't failed a course before this. Can I continue?", LLAMA_3_8B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: I have an average of 54% and I have failed two courses this term. I haven't failed a course before this. Can I continue? 

result: Hi there! Sorry to hear that you've had a tough term. With a sessional average of 54% and having failed two courses, I'd like to clarify your academic standing. 

According to the Faculty of Science's academic performance review and continuation policy, if you have a sessional average of at least 50.0% but less than 55.0%, having failed one or more courses, you will be assigned Failed Standing. 

In your case, since you've failed two courses, you will likely be assigned Failed Standing. Unfortunately, this means you will be required to withdraw from the University entirely. 

However, don't worry! You can still continue your studies at UBC. You'll need to consult the Advancement Regulations and Withdrawal and Re-admission policies to understand the next steps. 

Remember, it's okay to face setbacks, and it's great that you're taking proactive steps t

In [16]:
response = answer_prompt("My average was 40%, what happens now?", LLAMA_3_8B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: My average was 40%, what happens now? 

result: Hi there! Sorry to hear that your average is 40%. If you're concerned about your academic standing, I recommend reaching out to the Science Advising team for guidance. They can help you explore options, such as meeting with an advisor or accessing resources to support your academic success. Additionally, you can also review the UBC's academic standing policy to understand the implications of your current average. Let's work together to find a solution that suits your needs! 

source_documents: [{'url': 'https://science.ubc.ca/students/advising', 'text': "1. Join the Science Advising line. \n  2. Review the final confirmation screen carefully.\n  3. When it's your turn, you'll get a text with instructions.", 'doc_relate': ' No, the document does not relate to the query. The document appears to be about the process for drop-in advising, whereas the query is about what happens when someone has an average of 40%.'}, {'url': 'https://sc

In [18]:
response = answer_prompt("I missed my exam because I had covid, what do I do?", LLAMA_3_8B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: I missed my exam because I had covid, what do I do? 

result: Sorry to hear that you missed your exam due to COVID-19! 

According to the UBC Science website, if you're sick, it's important to stay home and not attend your exam. If you're unable to write the exam, contact Science Advising as soon as possible. You may need to provide a medical note to support your request for a deferred exam. Additionally, if you're experiencing an exam hardship (e.g., three exams scheduled within 24 hours), notify your instructor immediately. 

Please reach out to Science Advising for guidance on next steps. They're here to support you! 

source_documents: [{'url': 'https://science.ubc.ca/students/exam-issues', 'text': 'If you’re sick, it’s important that you stay home – no matter what you think you may be sick with (e.g., cold, flu, other). See the information under “I missed an exam. What do I do?” for next steps.', 'doc_relate': ' Yes, the document relates to the query. The document provides 

In [19]:
response = answer_prompt("I slept through my exam, what do I do?", LLAMA_3_8B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: I slept through my exam, what do I do? 

result: Oh no, sorry to hear that! If you slept through your exam, it's essential to act quickly. First, reach out to Science Advising as soon as possible. They'll guide you through the process and help you figure out what to do next. If you're feeling unwell, be sure to get a medical note, as documentation is required for a deferred exam. Additionally, if you're experiencing an exam hardship (e.g., three final exams within 24 hours), notify your instructor right away. Remember, it's crucial to take immediate action to minimize the impact on your academic progress. 

source_documents: [{'url': 'https://science.ubc.ca/students/exam-issues', 'text': 'If you’re sick, it’s important that you stay home – no matter what you think you may be sick with (e.g., cold, flu, other). See the information under “I missed an exam. What do I do?” for next steps.', 'doc_relate': " Yes, the document relates to the query. The document provides information on 

In [20]:
response = answer_prompt("Does ENGL 110 count for the arts requirement?", LLAMA_3_70B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: Does ENGL 110 count for the arts requirement? 

result: I'm happy to help you with that! However, I don't have enough information to provide a specific answer. The provided context doesn't mention ENGL 110 or its relation to the arts requirement. Could you please provide more context or clarify which program or faculty you're referring to? I'll do my best to assist you. 

source_documents: [{'url': 'https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-arts/bachelor-fine-arts/academic-advising', 'text': 'The Arts Academic Advising office, located in the Buchanan complex, Block D, Room D111 (1866 Main Mall), is open throughout the year: Monday-Friday, from 9:00 am-4:30 pm, for in-person, telephone (604-822-4028), and email (arts.advisor@ubc.ca) inquiries. Drop-in advising and pre-booked appointments are available daily. Students should consult the Arts Undergraduates website prior to contacting the Advising office for an inquiry or to request an advising appoin

In [21]:
response = answer_prompt("Can I take PSYC 120 for the arts requirement?", LLAMA_3_70B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: Can I take PSYC 120 for the arts requirement? 

result: I'd be happy to help you with that! However, I need a bit more information to give you a accurate answer. Could you please tell me what program you're in (e.g., Bachelor of Fine Arts, Bachelor of Arts, etc.) and what faculty you're a part of (e.g., Faculty of Arts, Faculty of Science, etc.)? That way, I can give you the best advice possible! 

source_documents: [{'url': 'https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-arts/bachelor-fine-arts/academic-advising', 'text': 'The Arts Academic Advising office, located in the Buchanan complex, Block D, Room D111 (1866 Main Mall), is open throughout the year: Monday-Friday, from 9:00 am-4:30 pm, for in-person, telephone (604-822-4028), and email (arts.advisor@ubc.ca) inquiries. Drop-in advising and pre-booked appointments are available daily. Students should consult the Arts Undergraduates website prior to contacting the Advising office for an inquiry or to

In [12]:
response = answer_prompt("Does physics 100 count for the arts requirement?", LLAMA_3_70B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: Does physics 100 count for the arts requirement? 

result: I'm happy to help you out! According to the UBC calendar, the Arts Requirement is specific to courses offered by the Faculty of Arts. Physics 100 is a Science course, so it wouldn't count towards the Arts Requirement. You'll need to take courses from the Faculty of Arts, such as those in the humanities, social sciences, or languages, to fulfill this requirement. If you have any more questions or need help finding suitable Arts courses, feel free to ask! 

source_documents: [{'url': 'https://science.ubc.ca/students/minors', 'text': 'Students should design a course of studies for their proposed Arts Minor, which must be approved by Science Advising by the end of Year 3.18-credit Minor in Arts  \nStudents must submit an application to Science Advising for review.30-credit Minor in Arts  \nStudents need approval from the appropriate department/program advisor in Arts (not the Arts Academic Advising Office) before submitting 

In [223]:
response = answer_prompt("CPSC 110 waitlist is full, what do I do?", LLAMA_3_70B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: CPSC 110 waitlist is full, what do I do? 

result: Oh no, I'm so sorry to hear that the CPSC 110 waitlist is full! Don't worry, there's still hope! According to our courses FAQ, a lot of registration changes happen in the first week or so of school, sometimes even right up until the add/drop deadline. You might want to try checking the course registration regularly to see if any spots open up. Additionally, you can also try reaching out to your instructor or academic advisor to see if they have any advice or can offer any assistance. Remember, we're all here to help you achieve your academic goals! If you have any more questions or need further guidance, feel free to connect with Science Advising. 

source_documents: [{'url': 'https://science.ubc.ca/students/courses-faq', 'text': 'As noted above, we can’t put you in a course if it’s full; however, if you’ve encountered another difficulty that is not resolved with the above steps, connect with us right away.', 'doc_relate': ' Yes

In [224]:
response = answer_prompt("I am in second year major chemistry, do I have to take MATH 221?", LLAMA_3_70B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: I am in second year major chemistry, do I have to take MATH 221? 

result: Hi there! I'm happy to help you with your question. According to the UBC calendar, there is no specific requirement for MATH 221 in the second year of the Major in Chemistry program. However, I would recommend checking your program requirements and consulting with an academic advisor to ensure you're meeting all the necessary requirements for your degree. Additionally, you may want to consider taking MATH 221 if you're interested in pursuing a more theoretical or computational aspect of chemistry, as it can be a useful skillset to have. But ultimately, it's not a required course for your program. Hope this helps! 

source_documents: [{'url': 'https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-science/bachelor-science/mathematics#5464', 'text': '- MATH 200 (or 226) ( Students obtaining 68% or higher in MATH 226 may replace MATH 220 with 3 credits of electives. In order to be promoted 

In [225]:
response = answer_prompt("I am in second year major chemistry, do I need to take any biology courses?", LLAMA_3_70B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: I am in second year major chemistry, do I need to take any biology courses? 

result: Ah, hi there! As a friendly UBC student advising assistant, I'm more than happy to help you out. According to the UBC calendar, the chemistry major doesn't explicitly require any biology courses. However, it's essential to note that some chemistry courses might have biology prerequisites or co-requisites. I would recommend checking the course descriptions and prerequisites for the specific chemistry courses you're interested in taking to ensure you meet the requirements. Additionally, if you're considering a minor or specialization that involves biology, you might need to take biology courses. It's always a good idea to consult with a Science advisor or your departmental advisor to get personalized advice tailored to your academic goals. They can help you create a plan that suits your needs. Would you like me to help you find the contact information for the Science advising team or your departm

In [226]:
response = answer_prompt("Can I take CHEM 233 instead of CHEM 203?", LLAMA_3_70B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: Can I take CHEM 233 instead of CHEM 203? 

result: According to the UBC calendar, students with CHEM 235 and a score of 76% or higher in CHEM 233 may apply for admission to the Honours (0213): Chemistry (CHEM) specialization and will be allowed to use CHEM 233 and 235 in place of CHEM 203. So, yes, you can take CHEM 233 instead of CHEM 203, but you need to meet the specific requirements mentioned above. 

source_documents: [{'url': 'https://vancouver.calendar.ubc.ca/faculties-colleges-and-schools/faculty-science/bachelor-science/chemistry#5154', 'text': 'CHEM 123 is the normal prerequisite for admission to Chemistry specializations. Students with credit only for Chemistry 11 take CHEM 111 before taking CHEM 123, whereas those with credit for Chemistry 12 take CHEM 121. In addition, students applying for entrance to Chemistry specializations must have completed MATH 100 and MATH 101 (or their equivalents).', 'doc_relate': ' Yes, the document is relevant to the query. The document

In [21]:
response = answer_prompt("What are the specializations in the Faculty of Science?", LLAMA_3_8B)

for key, value in response.items():
    print(f"{key}: {value} \n")

query: What are the specializations in the Faculty of Science? 

result: The Faculty of Science at UBC offers a wide range of specializations across various departments. Some examples include:

* Biological Sciences: Biochemistry, Biology, Biophysics, Microbiology, and Neuroscience
* Chemistry: Analytical Chemistry, Biochemistry, Inorganic Chemistry, Organic Chemistry, Physical Chemistry, and Theoretical Chemistry
* Earth and Ocean Sciences: Earth Sciences, Ocean Sciences, and Planetary Science
* Mathematics: Pure Mathematics, Applied Mathematics, and Statistics
* Physics and Astronomy: Physics, Astronomy, and Theoretical Physics

Please note that specializations may vary depending on the department and program. For more information, I recommend visiting the Faculty of Science's website or consulting with a Science advisor. 

source_documents: [{'url': 'https://science.ubc.ca/students/advising', 'text': "If a medical or personal issue is affecting your academic life see us as soon as p